In [1]:
from PIL import Image
import requests
from io import BytesIO
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob
import random
import shutil

In [2]:
# get images from pf_ext
labels = os.path.join('Pf_ext_01.json')

count = 1
phase_count = {
    "train": 1,
    "test": 1
}
phase_count["train"] = 1
phase_count["test"] = 1

phase = "train"
degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('images',name+".jpg"))
            f = open(os.path.join('labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "2":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "1":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1
            for deg in degs:
                name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
                image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (((x2-x1)/2)+x1)/aug_w
                    bb_yc = (((y2-y1)/2)+y1)/aug_h
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    bboxs = bboxs[1:]
                    im_aug = Image.fromarray(image_aug)
                    im_aug.save(os.path.join('images',name+".jpg"))
                    f = open(os.path.join('labels',name+".txt"),'w')
                    f.write(bboxs)
                    f.close()
                    print("{} {}".format(phase_count[phase],deg))
                    phase_count[phase] += 1

1 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015250.jpg
pf-train-0000000001
2 45
3 90
4 135
5 180
6 225
7 270
8 315
2 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015251.jpg
pf-train-0000000009
10 45
11 90
12 135
13 180
14 225
15 270
16 315
3 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015252.jpg
pf-train-0000000017
18 45
19 90
20 135
21 180
22 225
23 270
24 315
4 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015253.jpg
pf-train-0000000025
26 45
27 90
28 135
29 180
30 225
31 270
32 315
5 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015254.jpg
pf-train-0000000033
34 45
35 90
36 135
37 180
38 225
39 270
40 315
6 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015255.jpg
pf-train-0000000041
42 45
43 90
44 135
45 180
46 225
47 270
48 315
7 https://raw.githubusercontent.com/RTANC/malaria_dataset/m

402 45
403 90
404 135
405 180
406 225
407 270
408 315
52 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015301.jpg
pf-train-0000000409
410 45
411 90
412 135
413 180
414 225
415 270
416 315
53 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015302.jpg
pf-train-0000000417
418 45
419 90
420 135
421 180
422 225
423 270
424 315
54 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015303.jpg
pf-train-0000000425
426 45
427 90
428 135
429 180
430 225
431 270
432 315
55 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015304.jpg
pf-train-0000000433
434 45
435 90
436 135
437 180
438 225
439 270
440 315
56 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015305.jpg
pf-train-0000000441
442 45
443 90
444 135
445 180
446 225
447 270
448 315
57 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015306.jpg
pf-train-0000000449
450 

802 45
803 90
804 135
805 180
806 225
807 270
808 315
102 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015351.jpg
pf-train-0000000809
810 45
811 90
812 135
813 180
814 225
815 270
816 315
103 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015352.jpg
pf-train-0000000817
818 45
819 90
820 135
821 180
822 225
823 270
824 315
104 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015353.jpg
pf-train-0000000825
826 45
827 90
828 135
829 180
830 225
831 270
832 315
105 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015354.jpg
pf-train-0000000833
834 45
835 90
836 135
837 180
838 225
839 270
840 315
106 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015355.jpg
pf-train-0000000841
842 45
843 90
844 135
845 180
846 225
847 270
848 315
107 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015356.jpg
pf-train-000000084

In [3]:
print(count)
print(phase_count["train"])

118
937


In [4]:
# get image from pf_01
labels = os.path.join('pf_01.json')
count = 1
degs = [45,90,135,180,225,270,315]
with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('images',name+".jpg"))
            f = open(os.path.join('labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1
            for deg in degs:
                name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
                image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (x1/aug_w) + (bb_w/2)
                    bb_yc = (y1/aug_h) + (bb_h/2)
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    bboxs = bboxs[1:]
                    im_aug = Image.fromarray(image_aug)
                    im_aug.save(os.path.join('images',name+".jpg"))
                    f = open(os.path.join('labels',name+".txt"),'w')
                    f.write(bboxs)
                    f.close()
                    print("{} {}".format(phase_count[phase],deg))
                    phase_count[phase] += 1

1 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i01j17.jpg
938 45
939 90
940 135
941 180
942 225
943 270
944 315
2 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i01j21.jpg
946 180
3 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i01j22.jpg
4 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i02j15.jpg
949 45
950 90
951 135
952 180
953 225
954 270
955 315
5 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i02j16.jpg
957 45
958 90
959 135
960 180
961 225
962 270
963 315
6 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i02j17.jpg
7 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i02j18.jpg
966 45
967 90
968 135
969 180
970 225
971 270
972 315
8 http

1287 315
52 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j35.jpg
1289 45
1290 90
1291 135
1292 180
1293 225
1294 270
1295 315
53 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j36.jpg
1297 45
1298 90
1299 135
1300 180
1301 225
1302 270
1303 315
54 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j37.jpg
1305 45
1306 90
1307 135
1308 180
1309 225
1310 270
1311 315
55 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j38.jpg
1313 45
1314 90
1315 135
1316 180
1317 225
1318 270
1319 315
56 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j39.jpg
1321 45
1322 90
1323 135
1324 180
1325 225
1326 270
1327 315
57 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j40.jpg
1329 45
1330 90
1331

1633 45
1634 90
1635 135
1636 180
1637 225
1638 270
1639 315
103 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j35.jpg
1641 45
1642 90
1643 135
1644 180
1645 225
1646 270
1647 315
104 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j36.jpg
1649 45
1650 90
1651 135
1652 180
1653 225
1654 270
1655 315
105 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j37.jpg
1657 45
1658 90
1659 135
1660 180
1661 225
1662 270
1663 315
106 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j38.jpg
1665 45
1666 90
1667 135
1668 180
1669 225
1670 270
1671 315
107 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j39.jpg
1673 45
1674 90
1675 135
1676 180
1677 225
1678 270
1679 315
108 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_W

2011 45
2012 90
2013 135
2014 180
2015 225
2016 270
2017 315
151 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j27.jpg
2019 45
2020 90
2021 135
2022 180
2023 225
2024 270
2025 315
152 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j28.jpg
2027 45
2028 90
2029 135
2030 180
2031 225
2032 270
2033 315
153 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j29.jpg
2035 45
2036 90
2037 135
2038 180
2039 225
2040 270
2041 315
154 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j30.jpg
2043 45
2044 90
2045 135
2046 180
2047 225
2048 270
2049 315
155 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j31.jpg
2051 45
2052 90
2053 135
2054 180
2055 225
2056 270
2057 315
156 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_W

2384 315
199 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j16.jpg
2386 45
2387 90
2388 135
2389 180
2390 225
2391 270
2392 315
200 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j17.jpg
2394 45
2395 90
2396 135
2397 180
2398 225
2399 270
2400 315
201 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j18.jpg
2402 45
2403 180
2404 225
202 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j19.jpg
2406 45
2407 90
2408 135
2409 180
2410 225
2411 270
2412 315
203 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j20.jpg
2414 45
2415 90
2416 135
2417 180
2418 225
2419 270
2420 315
204 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j22.jpg
2422 45
2423 90
2424 135
2425 180
2426 225
2427 2

...

4606 45
4607 90
4608 135
4609 180
4610 225
4611 270
4612 315
490 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j21.jpg
4614 45
4615 90
4616 135
4617 180
4618 225
4619 270
4620 315
491 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j22.jpg
4622 45
4623 90
4624 135
4625 180
4626 225
4627 270
4628 315
492 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j23.jpg
4630 45
4631 90
4632 135
4633 180
4634 225
4635 270
4636 315
493 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j24.jpg
4638 45
4639 90
4640 135
4641 180
4642 225
4643 270
4644 315
494 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j25.jpg
4646 45
4647 90
4648 135
4649 180
4650 225
4651 270
4652 315
495 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_W

4981 45
4982 90
4983 135
4984 180
4985 225
4986 270
4987 315
538 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j71.jpg
4989 45
4990 90
4991 135
4992 180
4993 225
4994 270
4995 315
539 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j16.jpg
4997 45
4998 90
4999 180
5000 225
5001 270
540 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j18.jpg
5003 45
5004 90
5005 135
5006 180
5007 225
5008 270
5009 315
541 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j19.jpg
5011 45
5012 90
5013 135
5014 180
5015 225
5016 270
5017 315
542 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j20.jpg
5019 45
5020 90
5021 135
5022 180
5023 225
5024 270
5025 315
543 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_

5355 45
5356 90
5357 135
5358 180
5359 225
5360 270
5361 315
586 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j67.jpg
5363 45
5364 90
5365 135
5366 180
5367 225
5368 270
5369 315
587 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j68.jpg
5371 45
5372 90
5373 135
5374 180
5375 225
5376 270
5377 315
588 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j69.jpg
5379 45
5380 90
5381 135
5382 180
5383 225
5384 270
5385 315
589 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j70.jpg
5387 45
5388 90
5389 135
5390 180
5391 225
5392 270
5393 315
590 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j71.jpg
5395 45
5396 90
5397 135
5398 180
5399 225
5400 270
5401 315
591 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_W

5735 45
5736 90
5737 135
5738 180
5739 225
5740 270
5741 315
634 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j60.jpg
5743 45
5744 90
5745 135
5746 180
5747 225
5748 270
5749 315
635 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j61.jpg
5751 45
5752 90
5753 135
5754 180
5755 225
5756 270
5757 315
636 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j62.jpg
5759 45
5760 90
5761 135
5762 180
5763 225
5764 270
5765 315
637 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j63.jpg
5767 45
5768 90
5769 135
5770 180
5771 225
5772 270
5773 315
638 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j64.jpg
5775 45
5776 90
5777 135
5778 180
5779 225
5780 270
5781 315
639 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_W

6108 180
6109 225
6110 270
6111 315
682 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j51.jpg
6113 45
6114 90
6115 135
6116 180
6117 225
6118 270
6119 315
683 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j52.jpg
6121 45
6122 90
6123 135
6124 180
6125 225
6126 270
6127 315
684 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j53.jpg
6129 45
6130 90
6131 135
6132 180
6133 225
6134 270
6135 315
685 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j55.jpg
6137 45
6138 90
6139 135
6140 180
6141 225
6142 270
6143 315
686 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j56.jpg
6145 45
6146 90
6147 135
6148 180
6149 225
6150 270
6151 315
687 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extende

In [5]:
print(count)
print(phase_count["train"])

697
6232


In [6]:
# get image from pf_test
labels = os.path.join('pf_test.json')

count = 1
phase_count = {
    "test": 1
}

phase = "test"
degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('images',name+".jpg"))
            f = open(os.path.join('labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1

1 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/16.jpg
pf-test-0000000001
2 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/23.jpg
pf-test-0000000002
3 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/3.jpg
pf-test-0000000003
4 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/31.jpg
pf-test-0000000004
5 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/6.jpg
pf-test-0000000005
6 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/60.jpg
pf-test-0000000006
7 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/63.jpg
pf-test-0000000007
8 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/66.jpg
pf-test-0000000008
9 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/68.jpg
pf-test-0000000009
10 https://r

In [7]:
# get image from pf_02 (Test set)
labels = os.path.join('pf_02.json')

count = 1
phase_count = {
    "test": 11
}

phase = "test"
degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0 and count <= 100:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('images',name+".jpg"))
            f = open(os.path.join('labels',name+".txt"),'w')
#             rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "2":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
#                     rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
#             bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            phase_count[phase] += 1
        count += 1

2 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/02/Data20200115-02_Wholeslide_Default_Extended_i01j02.jpg
pf-test-0000000011
4 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/02/Data20200115-02_Wholeslide_Default_Extended_i01j04.jpg
pf-test-0000000012
5 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/02/Data20200115-02_Wholeslide_Default_Extended_i01j05.jpg
pf-test-0000000013
6 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/02/Data20200115-02_Wholeslide_Default_Extended_i01j06.jpg
pf-test-0000000014
7 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/02/Data20200115-02_Wholeslide_Default_Extended_i01j07.jpg
pf-test-0000000015
8 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/02/Data20200115-02_Wholeslide_Default_Extended_i01j08.jpg
pf-test-0000000016
9 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/02/Data20200115-02_Wholeslide_Default_Extended_i01j09.jpg
pf-test-0000000017
10 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/02/Data20200115-02_Wholeslide_Default_Extend

In [ ]:
# summary training set
# files = glob.glob("images/pf-train-*.jpg")
# count = 0
# count_class = [0,0]

# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     lines = list(f)
#     f.close()
#     print(img)
#     for line in lines:
#         print(line)
#         class_id, xc, yc, wb, hb = line.split()
#         count_class[int(class_id)] += 1
#     count += 1
# print(count)
# print(count_class)

In [ ]:
# clear empty box
# files = glob.glob("images/*")
# count = 0
# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     txt = list(f)
#     f.close()
#     if len(txt) == 0:
#         print(img)
#         os.remove("labels/"+name+".txt")
#         os.remove(file)
#         count += 1
# print(count)

In [ ]:
# labels = os.path.join('Pf','pf_01.json')
# count = 0
# with open(labels,'r') as json_file:
#     data = json.load(json_file)
#     images = data["_via_img_metadata"]
#     for key in images:
#         url = images[key]["filename"]
#         if count >= 120 and count <= 240:
#             name = "{:010d}".format(count)
#             print("{} {}".format(count,url))
#             response = requests.get(url)
#             img = Image.open(BytesIO(response.content))
#             w, h = img.size
#             img.save(os.path.join('data','samples',name+".jpg"))
#         count += 1
    

In [ ]:
# count = 9784
# for i in range(count,12583):
#     name = "{:010d}".format(i)
#     print(name)
#     os.remove(os.path.join("data/custom/images/",name+".jpg"))
#     os.remove(os.path.join("data/custom/labels/",name+".txt"))